In [1]:
#%env NEURO_RAS_BACKEND=sql

In [1]:
import sys
from typing import Iterable
import warnings

import nibabel as nib
from nilearn import datasets, image, plotting
import numpy as np
import pandas as pd


import xml.etree.ElementTree as ET
import json
from neurolang.frontend import NeurolangPDL
from rdflib import RDFS

/Users/gzanitti/.local/lib/python3.8/site-packages/scikit_learn-0.23.2-py3.8-macosx-10.9-x86_64.egg/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.feature_selection.univariate_selection module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
def parse_region(elem, id_2_num, father=None, triples=[]):
    name = elem['name']
    if 'labelIndex' in elem:
        if elem['labelIndex'] is not None:
            index = int(elem['labelIndex'])
            if index in id_2_num:
                num = id_2_num[index]
                triples.append((name, num))
        
    for c in elem['children']:
        parse_region(c, id_2_num, father=name, triples=triples)
        
    return triples

In [20]:
# Ontology
julich_ontology_l = datasets.utils._fetch_files(
    datasets.utils._get_dataset_dir('julich_ontology'),
    [
        (
            'julich_ontology_l.xml',
            'https://raw.githubusercontent.com/NeuroLang/neurolang_data/main/Julich-Brain/WB/22/MPM/'
            'JulichBrain_MPMAtlas_l_N10_nlin2Stdicbm152asym2009c_publicDOI_3f6407380a69007a54f5e13f3c1ba2e6.xml',
            {'move': 'julich_ontology_l.xml'}
        )
    ]
)[0]

julich_ontology_r = datasets.utils._fetch_files(
    datasets.utils._get_dataset_dir('julich_ontology'),
    [
        (
            'julich_ontology_r.xml',
            'https://raw.githubusercontent.com/NeuroLang/neurolang_data/main/Julich-Brain/WB/22/MPM/'
            'JulichBrain_MPMAtlas_l_N10_nlin2Stdicbm152asym2009c_publicDOI_3f6407380a69007a54f5e13f3c1ba2e6.xml',
            {'move': 'julich_ontology_r.xml'}
        )
    ]
)[0]

jubrain_ontology = datasets.utils._fetch_files(
    datasets.utils._get_dataset_dir('julich_ontology'),
    [
        (
            'jubrain_ontology.xml',
            'https://raw.githubusercontent.com/NeuroLang/neurolang_data/main/Julich-Brain/WB/22/jubrain-ontology_22.json',
            {'move': 'jubrain_ontology.xml'}
        )
    ]
)[0]

tree = ET.parse(julich_ontology_l)

id_2_num = {}
for a in tree.iter():
    if a.tag == 'Structure':
        num = int(a.attrib['grayvalue'])
        id_ = int(a.attrib['id'])
        id_2_num[id_] = num

tree = ET.parse(julich_ontology_r)

for a in tree.iter():
    if a.tag == 'Structure':
        num = int(a.attrib['grayvalue'])
        id_ = int(a.attrib['id'])
        id_2_num[id_] = num


with open(jubrain_ontology) as f:
    data = json.load(f)

regions = data['properties']['regions']
for elem in regions:
    triples = parse_region(elem, id_2_num)
    
    #for n, r in [
    #    (13, 'GapMap Frontal-I (GapMap)'),
    #    (32, 'GapMap Frontal-to-Occipital (GapMap)'),
    #    (59, 'GapMap Temporal-to-Parietal (GapMap)'),
    #    (89, 'GapMap Frontal-II (GapMap)'),
    #    (95, 'GapMap Frontal-to-Temporal (GapMap)')
    #]:
    #    triples.append((r, n))
        
    f.close()   
    regions = pd.DataFrame(triples, columns=['r_name', 'r_number']).astype({'r_number': 'int32'}).sort_values('r_number')
    regions.drop_duplicates(inplace=True)

In [21]:
regions2 = regions.copy()
regions2['r_number'] = regions2['r_number'] + 1000
regions2['hemis'] = 'l'
regions['hemis'] = 'r'

regions = pd.concat((regions, regions2))

In [22]:
## Atlas
mni_t1 = nib.load(datasets.fetch_icbm152_2009()['t1'])
mni_t1_4mm = image.resample_img(mni_t1, np.eye(3) * 4)

wb22_l = datasets.utils._fetch_files(
    datasets.utils._get_dataset_dir('julich'),
    [
        (
            'wb22_l.nii.gz',
            'https://github.com/NeuroLang/neurolang_data/raw/main/Julich-Brain/WB/22/MPM/'
            'JulichBrain_MPMAtlas_l_N10_nlin2Stdicbm152asym2009c_publicDOI_3f6407380a69007a54f5e13f3c1ba2e6.nii.gz',
            {'move': 'wb22_l.nii.gz'}
        )
    ]
)[0]

wb22_r = datasets.utils._fetch_files(
    datasets.utils._get_dataset_dir('julich'),
    [
        (
            'wb22_r.nii.gz',
            'https://github.com/NeuroLang/neurolang_data/raw/main/Julich-Brain/WB/22/MPM/'
            'JulichBrain_MPMAtlas_r_N10_nlin2Stdicbm152asym2009c_publicDOI_14622b49a715338ce96e96611d395646.nii.gz',
            {'move': 'wb22_r.nii.gz'}
        )
    ]
)[0]

img_r = image.load_img(wb22_r)
img_l = image.load_img(wb22_l)
img_l_data = img_l.get_fdata()
img_r_data = img_r.get_fdata()
img_l_unmaskes = np.nonzero(img_l_data)

for v in zip(*img_l_unmaskes):
    value = img_l_data[v[0]][v[1]][v[2]]
    ex_value = img_r_data[v[0]][v[1]][v[2]]
    if ex_value == 0:
        img_r_data[v[0]][v[1]][v[2]] = value + 1000
    
conc_img = nib.spatialimages.SpatialImage(img_r_data, img_r.affine)

conc_img = image.resample_img(
    conc_img, mni_t1_4mm.affine, interpolation='nearest'
)

conc_img_data = conc_img.get_fdata()
conc_img_unmaskes = np.nonzero(conc_img_data)

julich_brain = []
for v in zip(*conc_img_unmaskes):
    julich_brain.append((v[0], v[1], v[2], conc_img_data[v[0]][v[1]][v[2]]))

In [23]:
# NeuroSynth
ns_database_fn, ns_features_fn = datasets.utils._fetch_files(
    datasets.utils._get_dataset_dir('neurosynth'),
    [
        (
            'database.txt',
            'https://github.com/neurosynth/neurosynth-data/raw/master/current_data.tar.gz',
            {'uncompress': True}
        ),
        (
            'features.txt',
            'https://github.com/neurosynth/neurosynth-data/raw/master/current_data.tar.gz',
            {'uncompress': True}
        ),
    ]
)

ns_database = pd.read_csv(ns_database_fn, sep=f'\t')
ijk_positions = (
    nib.affines.apply_affine(
        np.linalg.inv(mni_t1_4mm.affine),
        ns_database[['x', 'y', 'z']]
    ).astype(int)
)
ns_database['i'] = ijk_positions[:, 0]
ns_database['j'] = ijk_positions[:, 1]
ns_database['k'] = ijk_positions[:, 2]

ns_features = pd.read_csv(ns_features_fn, sep=f'\t')
ns_terms = (
    pd.melt(
            ns_features,
            var_name='term', id_vars='pmid', value_name='TfIdf'
       )
    .query('TfIdf > 1e-3')[['pmid', 'term']]
)
ns_docs = ns_features[['pmid']].drop_duplicates()

In [24]:
# CogAt
cogAt = datasets.utils._fetch_files(
    datasets.utils._get_dataset_dir('CogAt'),
    [
        (
            'cogat.xml',
            'http://data.bioontology.org/ontologies/COGAT/download?'
            'apikey=8b5b7825-538d-40e0-9e9e-5ab9274a9aeb&download_format=rdf',
            {'move': 'cogat.xml'}
        )
    ]
)[0]

In [25]:
nl = NeurolangPDL()
nl.load_ontology(cogAt)

@nl.add_symbol
def agg_max(i: Iterable) -> float:
    return np.max(i)

@nl.add_symbol
def mean(iterable: Iterable) -> float:
    return np.mean(iterable)


@nl.add_symbol
def std(iterable: Iterable) -> float:
    return np.std(iterable)


part_of = nl.new_symbol(name='http://www.obofoundry.org/ro/ro.owl#part_of')
subclass_of = nl.new_symbol(name=str(RDFS.subClassOf))
label = nl.new_symbol(name=str(RDFS.label))
hasTopConcept = nl.new_symbol(name='http://www.w3.org/2004/02/skos/core#hasTopConcept')

@nl.add_symbol
def word_lower(name: str) -> str:
    return name.lower()

from sklearn.model_selection import KFold
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

ns_doc_folds = pd.concat(
    ns_docs.iloc[train].assign(fold=[i] * len(train))
    for i, (train, _) in enumerate(kfold.split(ns_docs))
)
doc_folds = nl.add_tuple_set(ns_doc_folds, name='doc_folds')


activations = nl.add_tuple_set(ns_database.values, name='activations')
terms = nl.add_tuple_set(ns_terms.values, name='terms')
docs = nl.add_uniform_probabilistic_choice_over_set(
        ns_docs.values, name='docs'
)

terms_det = nl.add_tuple_set(
        ns_terms.term.unique(), name='terms_det'
)

j_brain = nl.add_tuple_set(
    julich_brain,
    name='julich_brain'
)

j_regions = nl.add_tuple_set(
    regions.values,
    name='julich_regions'
)

In [26]:
#%env NEURO_SQLA_DIALECT=neurolang.db

In [27]:
'''with nl.scope as e:

    e.ontology_terms[e.onto_name] = (
        hasTopConcept[e.uri, e.cp] &
        label[e.uri, e.onto_name]
    )

    e.lower_terms[e.term] = (
        e.ontology_terms[e.onto_term] &
        (e.term == word_lower[e.onto_term])
    )
    
    e.filtered_terms[e.d, e.t] = (
        e.terms[e.d, e.t] &
        e.lower_terms[e.t]
    )

    e.act_regions[e.t, e.id_region] = (
        e.julich_brain[e.i, e.j, e.k, e.id_region] &
        e.activations[
            e.d, ..., ..., ..., ..., 'MNI', ..., ..., ..., ...,
            ..., ..., ..., e.i, e.j, e.k
        ] &
        e.filtered_terms[e.d, e.t]
    )

    e.no_act_regions[e.t, e.id_region] = (
        e.julich_regions[..., e.id_region] &
        e.filtered_terms[..., e.t] &
        ~(e.act_regions[e.t, e.id_region])
    )

    e.term_prob[e.t, e.fold, e.PROB[e.t, e.fold]] = (
       (e.docs[e.d] & e.doc_folds[e.d, e.fold] & 
        e.filtered_terms[e.d, e.t] & e.act_regions[e.t, e.id]) // 
        (e.docs[e.d] &
        e.doc_folds[e.d, e.fold])  
    )


    e.no_term_prob[e.t, e.fold, e.PROB[e.t, e.fold]] = (
       (e.docs[e.d] & e.doc_folds[e.d, e.fold] & 
        e.filtered_terms[e.d, e.t] & e.no_act_regions[e.t, e.id]) // 
        (e.docs[e.d] & e.doc_folds[e.d, e.fold])  
    )


    e.term_prob_mean[e.t, e.mean(e.PROB)] = (
        e.term_prob[e.t, e.fold, e.PROB]    
    )

    e.term_prob_std[e.t, e.std(e.PROB)] = (
        e.term_prob[e.t, e.fold, e.PROB]
    )

    e.no_term_prob_mean[e.t, e.mean(e.PROB)] = (
        e.no_term_prob[e.t, e.fold, e.PROB]    
    )

    e.no_term_prob_std[e.t, e.std(e.PROB)] = (
        e.no_term_prob[e.t, e.fold, e.PROB]
    )

    e.prob_summary_stats_term[
        e.t, 
        e.act_mean, e.act_std, 
    ] = (
        e.term_prob_mean[e.t, e.act_mean] &
        e.term_prob_std[e.t, e.act_std] #&
    )

    e.prob_summary_stats_no_term[
        e.t, 
        e.no_act_mean, e.no_act_std, 
    ] = (
        e.no_term_prob_mean[e.t, e.no_act_mean] &
        e.no_term_prob_std[e.t, e.no_act_std] #&
    )


    res = nl.solve_all()
    '''

    #pss = res['prob_summary_stats_term'].as_pandas_dataframe()
    #pss['region'] = id_region

    #pss.to_hdf('neuro_paper_ri_term_results.hdf', key=f'region_{str(id_region)}')

    #pss = res['prob_summary_stats_no_term'].as_pandas_dataframe()
    #pss['region'] = id_region

    #pss.to_hdf('neuro_paper_ri_no_term_results.hdf', key=f'region_{str(id_region)}')

"with nl.scope as e:\n\n    e.ontology_terms[e.onto_name] = (\n        hasTopConcept[e.uri, e.cp] &\n        label[e.uri, e.onto_name]\n    )\n\n    e.lower_terms[e.term] = (\n        e.ontology_terms[e.onto_term] &\n        (e.term == word_lower[e.onto_term])\n    )\n    \n    e.filtered_terms[e.d, e.t] = (\n        e.terms[e.d, e.t] &\n        e.lower_terms[e.t]\n    )\n\n    e.act_regions[e.t, e.id_region] = (\n        e.julich_brain[e.i, e.j, e.k, e.id_region] &\n        e.activations[\n            e.d, ..., ..., ..., ..., 'MNI', ..., ..., ..., ...,\n            ..., ..., ..., e.i, e.j, e.k\n        ] &\n        e.filtered_terms[e.d, e.t]\n    )\n\n    e.no_act_regions[e.t, e.id_region] = (\n        e.julich_regions[..., e.id_region] &\n        e.filtered_terms[..., e.t] &\n        ~(e.act_regions[e.t, e.id_region])\n    )\n\n    e.term_prob[e.t, e.fold, e.PROB[e.t, e.fold]] = (\n       (e.docs[e.d] & e.doc_folds[e.d, e.fold] & \n        e.filtered_terms[e.d, e.t] & e.act_regions[e

In [28]:
with nl.scope as e:

    e.ontology_terms[e.onto_name] = (
    hasTopConcept[e.uri, e.cp] &
    label[e.uri, e.onto_name]
    )

    '''e.lower_terms[e.term] = (
        e.ontology_terms[e.onto_term] &
        (e.term == word_lower[e.onto_term])
    )

    e.filtered_terms[e.d, e.t] = (
        e.terms[e.d, e.t] &
        e.lower_terms[e.t]
    )'''

    f_term = nl.solve_all()

In [29]:
ontology_terms = f_term['ontology_terms'].as_pandas_dataframe()

In [30]:
l_terms = ontology_terms.onto_name.apply(lambda x: x.lower())

In [31]:
lower_terms = nl.add_tuple_set(l_terms.values, name='lower_terms')

In [32]:
with nl.scope as e:

    e.filtered_terms[e.d, e.t] = (
        e.terms[e.d, e.t] &
        e.lower_terms[e.t]
    )

    f_term = nl.solve_all()

In [33]:
filtered = f_term['filtered_terms'].as_pandas_dataframe()
filtered_terms = nl.add_tuple_set(filtered.values, name='filtered_terms')

terms_det = nl.add_tuple_set(
        filtered.t.unique(), name='terms_det'
)

In [38]:
regions = regions[regions.r_number != 103] #Remove amygdala
#regions = regions[regions.r_number > 1118]

In [40]:
import datetime

from tqdm.notebook import tqdm

for name, id_region, _ in tqdm(regions.values):
    print(f'{id_region} - {name}')
    
    with pd.HDFStore('neuro_paper_ri_no_term_probs_folds.hdf') as hdf:
        if f'/region_{str(id_region)}' in hdf.keys():
            end_time = datetime.datetime.now()
            print(f'{id_region} - {name}: {end_time - start_time}')
            print('--------------')
            continue
    
    with nl.scope as e:

        start_time = datetime.datetime.now()
        # insert code snippet here
        
        #e.ontology_terms[e.onto_name] = (
        #hasTopConcept[e.uri, e.cp] &
        #label[e.uri, e.onto_name]
        #)

        #e.lower_terms[e.term] = (
        #    e.ontology_terms[e.onto_term] &
        #    (e.term == word_lower[e.onto_term])
        #)

        #e.filtered_terms[e.d, e.t] = (
        #    e.terms[e.d, e.t] &
        #    e.lower_terms[e.t]
        #)

        e.act_regions[e.d, id_region] = (
            e.julich_brain[e.i, e.j, e.k, id_region] &
            e.activations[
                e.d, ..., ..., ..., ..., 'MNI', ..., ..., ..., ...,
                ..., ..., ..., e.i, e.j, e.k
            ]
        )
        
        e.no_act_regions[e.d, e.id] = (
            #e.filtered_terms[..., e.t] &
            ~(e.act_regions[e.d, e.id]) &
             e.doc_folds[e.d, ...] &
            e.julich_regions[..., e.id]
        )
        
        e.term_prob[e.t, e.fold, e.PROB[e.t, e.fold]] = (
            (
                e.filtered_terms[e.d, e.t]
            ) // (
                e.act_regions[e.d, id_region] &
                e.doc_folds[e.d, e.fold] &
                e.docs[e.d]
            )
        )

        e.no_term_prob[e.t, e.fold, e.PROB[e.t, e.fold]] = (
           (
                e.filtered_terms[e.d, e.t]
            ) // (
                e.no_act_regions[e.d, id_region] &
                e.doc_folds[e.d, e.fold] &
                e.docs[e.d]
            )
        )

        '''e.term_prob_mean[e.t, e.mean(e.PROB)] = (
            e.term_prob[e.t, e.fold, e.PROB]    
        )

        e.term_prob_std[e.t, e.std(e.PROB)] = (
            e.term_prob[e.t, e.fold, e.PROB]
        )
        
        e.no_term_prob_mean[e.t, e.mean(e.PROB)] = (
            e.no_term_prob[e.t, e.fold, e.PROB]    
        )

        e.no_term_prob_std[e.t, e.std(e.PROB)] = (
            e.no_term_prob[e.t, e.fold, e.PROB]
        )

        e.prob_summary_stats_term[
            e.t, 
            e.act_mean, e.act_std, 
        ] = (
            e.term_prob_mean[e.t, e.act_mean] &
            e.term_prob_std[e.t, e.act_std]
        )
        
        e.prob_summary_stats_no_term[
            e.t, 
            e.no_act_mean, e.no_act_std, 
        ] = (
            e.no_term_prob_mean[e.t, e.no_act_mean] &
            e.no_term_prob_std[e.t, e.no_act_std]
        )'''

        
        res = nl.solve_all()
        
        end_time = datetime.datetime.now()
        print(f'{id_region} - {name}: {end_time - start_time}')
        
        pss = res['term_prob'].as_pandas_dataframe()
        #pss['region'] = id_region
        
        pss.to_hdf('neuro_paper_ri_term_probs_folds.hdf', key=f'region_{str(id_region)}')
        
        pss = res['no_term_prob'].as_pandas_dataframe()
        #pss['region'] = id_region
        
        pss.to_hdf('neuro_paper_ri_no_term_probs_folds.hdf', key=f'region_{str(id_region)}')

  0%|          | 0/237 [00:00<?, ?it/s]

1 - Area TE 1.1 (HESCHL)
1 - Area TE 1.1 (HESCHL): 2:31:59.394409
--------------
2 - Area 3b (PostCG)
2 - Area 3b (PostCG): 2:31:59.563093
--------------
3 - Area hOc4la (LOC)
3 - Area hOc4la (LOC): 2:31:59.746073
--------------
4 - Area PF (IPL)
4 - Area PF (IPL): 2:31:59.967985
--------------
5 - Area hOc2 (V2, 18)
5 - Area hOc2 (V2, 18): 2:32:00.155334
--------------
6 - Area 6d2 (PreCG)
6 - Area 6d2 (PreCG): 2:32:00.350909
--------------
7 - Area Ig3 (Insula)
7 - Area Ig3 (Insula): 2:32:00.530474
--------------
8 - Area hOc4d (Cuneus)
8 - Area hOc4d (Cuneus): 2:32:00.730189
--------------
9 - Area Fo4 (OFC)
9 - Area Fo4 (OFC): 2:32:00.902704
--------------
10 - Area OP4 (POperc)
10 - Area OP4 (POperc): 2:32:01.076353
--------------
11 - Area 1 (PostCG)
11 - Area 1 (PostCG): 2:32:01.239201
--------------
12 - Ch 4 (Basal Forebrain)
12 - Ch 4 (Basal Forebrain): 2:32:01.416667
--------------
14 - Area hPO1 (POS)
14 - Area hPO1 (POS): 2:32:01.601653
--------------
15 - Dorsal Dentate N

112 - Area Fo6 (OFC): 2:32:23.900061
--------------
113 - Area hOc4v (LingG)
113 - Area hOc4v (LingG): 2:32:24.057936
--------------
114 - Area PFop (IPL)
114 - Area PFop (IPL): 2:32:24.207803
--------------
115 - Area p32 (pACC)
115 - Area p32 (pACC): 2:32:24.368911
--------------
116 - Area hOc1 (V1, 17, CalcS)
116 - Area hOc1 (V1, 17, CalcS): 2:32:24.522818
--------------
117 - Area Fo2 (OFC)
117 - Area Fo2 (OFC): 2:32:24.678967
--------------
118 - Area hIP7 (IPS)
118 - Area hIP7 (IPS): 2:32:24.844767
--------------
119 - Area TE 2.2 (STG)
119 - Area TE 2.2 (STG): 2:32:24.995466
--------------
120 - Area i29 (retrosplenial)
120 - Area i29 (retrosplenial): 2:32:25.150425
--------------
121 - Area 7PC (SPL)
121 - Area 7PC (SPL): 2:32:25.307030
--------------
122 - Area hOc4lp (LOC)
122 - Area hOc4lp (LOC): 2:32:25.463058
--------------
123 - Area TeI (STG)
123 - Area TeI (STG): 2:32:25.612100
--------------
124 - Area Fo3 (OFC)
124 - Area Fo3 (OFC): 2:32:25.761131
--------------
1001

1110 - SF (Amygdala)
1110 - SF (Amygdala): 0:07:00.278264
1111 - Area s24 (sACC)
1111 - Area s24 (sACC): 0:07:22.192192
1112 - Area Fo6 (OFC)
1112 - Area Fo6 (OFC): 0:07:08.350239
1113 - Area hOc4v (LingG)
1113 - Area hOc4v (LingG): 0:07:30.051403
1114 - Area PFop (IPL)
1114 - Area PFop (IPL): 0:07:23.667036
1115 - Area p32 (pACC)
1115 - Area p32 (pACC): 0:07:21.743200
1116 - Area hOc1 (V1, 17, CalcS)
1116 - Area hOc1 (V1, 17, CalcS): 0:07:26.436002
1117 - Area Fo2 (OFC)
1117 - Area Fo2 (OFC): 0:07:27.211502
1118 - Area hIP7 (IPS)
1118 - Area hIP7 (IPS): 0:07:25.191903
1119 - Area TE 2.2 (STG)
1119 - Area TE 2.2 (STG): 0:07:16.939179
1120 - Area i29 (retrosplenial)
1120 - Area i29 (retrosplenial): 0:07:12.023517
1121 - Area 7PC (SPL)
1121 - Area 7PC (SPL): 0:07:02.921792
1122 - Area hOc4lp (LOC)
1122 - Area hOc4lp (LOC): 0:07:01.218096
1123 - Area TeI (STG)
1123 - Area TeI (STG): 0:07:02.953347
1124 - Area Fo3 (OFC)
1124 - Area Fo3 (OFC): 0:07:02.556618


In [ ]:
a = res['prob_summary_stats_term'].as_pandas_dataframe().sort_values(['act_mean'], ascending=False).set_index('t')

In [ ]:
b = res['prob_summary_stats_no_term'].as_pandas_dataframe().sort_values(['no_act_mean'], ascending=False).set_index('t')

In [ ]:
(a.act_mean - b.no_act_mean).dropna().sort_values()

In [ ]:
a = res['term_prob'].as_pandas_dataframe().set_index(['t', 'fold'])
b = res['no_term_prob'].as_pandas_dataframe().set_index(['t', 'fold'])

In [ ]:
a.loc['memory']

In [ ]:
res['term_prob'].as_pandas_dataframe().t.nunique()

In [ ]:
a

In [ ]:
b

In [ ]:
a - b